In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
# import PIL
import tensorflow as tf
import pickle5 as pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import glob
import random

import datetime
off=0


In [66]:

ch1_mean, ch1_std = 970.4162, 1051.8454
ch2_mean, ch2_std = 1176.249, 1031.262
ch3_mean, ch3_std = 1273.2376, 1117.068
ch4_mean, ch4_std = 2266.9050, 1241.5509
means = [ch1_mean,ch2_mean,ch3_mean,ch4_mean]
vari = [ch1_std, ch2_std, ch3_std, ch4_std]
x_size, y_size = 230,230
def load_features(name):
    decoded = name.decode("UTF-8")
    if os.path.exists(decoded):
        with open(decoded, 'rb') as f:
            file = pickle.load(f)
            label = tf.strings.split(tf.strings.split(name, '/')[-1], '\\')[-2]
            if label == 'coal':
                label = [1,0,0]
            elif label == 'steel':
                label = [0,1,0]
            else: label = [0,0,1]
            if (file["B02"].shape[1]<230 or file["B02"].shape[2]<230): 
                print("oh oh, downloaded patch too smol:", decoded)
            B02, B03, B04, B08 = file['B02'][0][0:x_size,0:x_size], file['B03'][0][0:x_size,0:y_size], file['B04'][0][0:x_size,0:y_size],file['B08'][0][0:x_size,0:y_size]
            # B02, B03, B04, B08 = B02[off:-off,off:-off], B03[off:-off,off:-off], B04[off:-off,off:-off], B08[off:-off,off:-off] 
            B02 = (B02-ch1_mean)/ch1_std
            B03 = (B03-ch2_mean)/ch2_std
            B04 = (B04-ch3_mean)/ch3_std
            B08 = (B08-ch4_mean)/ch4_std
            features = np.array([ B02, B03, B04, B08]).transpose(1,2,0)
            # features = np.expand_dims(features, axis=0)
            return features, label
            # I have commented the line below but this should return
            # the features and the label in a one hot vector
            # return file['features'], file['targets']
    else:
        print("Something went wrong!")
        exit(-1)
        
def data_loader(filename):
    features, labels = tf.numpy_function(load_features, [filename], [tf.double, tf.int32])
    # features.set_shape((None, 242,242,4))
    # labels.set_shape(( 1))
    return features, labels

In [67]:
path = 'E:/Users/sentinel_industry/downloaded_aois/coal/'

pkl_files = glob.glob((path+"*.pickle"))


In [68]:
print(len(pkl_files))

2179


In [69]:
path_steel = 'E:/Users/sentinel_industry/downloaded_aois/steel/'
pkl_files_steel = glob.glob((path_steel+"*.pickle"))

In [70]:
path_steel_no_ind = 'E:/Users/sentinel_industry/downloaded_aois/steel/no_ind/'
pkl_files_steel_no_ind = glob.glob((path_steel_no_ind+"*.pickle"))

In [71]:
path_coal_no_ind = 'E:/Users/sentinel_industry/downloaded_aois/coal/no_ind/'
pkl_files_coal_no_ind = glob.glob((path_coal_no_ind+"*.pickle"))

In [72]:
print(len(pkl_files_steel))

1693


In [73]:
pkl_files = pkl_files + pkl_files_steel + pkl_files_steel_no_ind + pkl_files_coal_no_ind
random.Random(42).shuffle(pkl_files)
# pkl_files=pkl_files[0:100]

In [74]:
# from itertools import chain

# pkl_files = list(chain.from_iterable(zip(pkl_files, pkl_files_steel)))


In [75]:
print(len(pkl_files))

7829


In [76]:
dataset_len=len(pkl_files)

In [57]:
# pkl_files

In [58]:
# dataset = tf.data.Dataset.from_tensor_slices((pkl_files))

# def data_loader(filename):
#     features, labels = tf.numpy_function(load_features, [filename], [tf.double, tf.int32])
#     # features.set_shape((None, 242,242,4))
#     # labels.set_shape(( 1))
#     return features, labels

# dataset = dataset.map(data_loader)
#                # load_features, [filename], [tf.uint16, tf.string])))

In [77]:
dataset_len=len(pkl_files)

train_size = int(np.floor(0.8 * dataset_len))
valid_size = int(np.floor(0.1 * dataset_len))
test_size = int(np.floor(0.1 * dataset_len))

train = tf.data.Dataset.from_tensor_slices(pkl_files[0:train_size]).map(data_loader,num_parallel_calls=tf.data.AUTOTUNE)
valid = tf.data.Dataset.from_tensor_slices(pkl_files[train_size:train_size+valid_size]).map(data_loader,num_parallel_calls=tf.data.AUTOTUNE)
test = tf.data.Dataset.from_tensor_slices(pkl_files[train_size+valid_size:]).map(data_loader,num_parallel_calls=tf.data.AUTOTUNE)

In [78]:
# tf.data.experimental.save(
#     valid, 'savedata_valid')

# tf.data.experimental.save(
#     train, 'savedata_train')

In [2]:
valid = tf.data.experimental.load('savedata_valid')
train = tf.data.experimental.load('savedata_train')
train = train.shuffle(100,  reshuffle_each_iteration=True)

In [3]:
x_s = 230-2*off
y_s = 230-2*off
in_seed = 42
resize_rescale_augment = tf.keras.Sequential([
    layers.RandomRotation(0.2, seed=in_seed, input_shape=(x_s,y_s,4)),  # here, maybe try larger values
    layers.CenterCrop(140,140, ),
    layers.RandomCrop(120,120, seed=in_seed, ),  # to be changed with centercrop for data augmentation/ use centre first, then random?
    
    # layers.Rescaling(255),#,input_shape=(240,240,4)), # I think the data is scaled to 10000
    layers.RandomFlip("horizontal_and_vertical", seed=in_seed), 
    # norm_layer,
])

In [4]:

model = tf.keras.Sequential([
    resize_rescale_augment,
    layers.Conv2D(3, 5, padding='same', activation='tanh'), # this seems like a bit of a brute force approach to handing a 3 channel image to resnet, 
                                                            # maybe try changing the source so it accepts 4 channels?
    tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights=None,
    input_shape=(120, 120, 3),
    pooling=None,
    classes=3),
    layers.Flatten(), # does this make sense? or is there another way to get down to just three output dimensions?
    layers.Dense(3)
])

log_dir = "E:/Users/sentinel_industry/logs/train/threemod/inceptres" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = '120,140')

def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * tf.math.exp(-0.03)
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=50,verbose=1, restore_best_weights=True)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
model.build()
model.summary()

# now with normalisation
epochs=200
batch_size = 20
history = model.fit(
    train.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_data=valid.batch(batch_size, drop_remainder=True, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE),
    validation_steps=15,
    epochs=epochs,
    callbacks=[tensorboard_callback, lr_callback, es_callback]
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 120, 120, 4)       0         
                                                                 
 conv2d (Conv2D)             (None, 120, 120, 3)       303       
                                                                 
 inception_resnet_v2 (Functi  (None, 2, 2, 1536)       54336736  
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 6144)              0         
                                                                 
 dense (Dense)               (None, 3)                 18435     
                                                                 
Total params: 54,355,474
Trainable params: 54,294,930
Non-trainable params: 60,544
_____________________________________

In [5]:
model.save('saved_models/inceptionres')

INFO:tensorflow:Assets written to: saved_models/inceptionres\assets


INFO:tensorflow:Assets written to: saved_models/inceptionres\assets
